# Connect to the simulator

In [1]:
import numpy as np
import sys

sys.path.append('E:/UADY/CARLA/CARLA_Latest/WindowsNoEditor/PythonAPI/carla')

import carla

client = carla.Client('localhost', 2000)

# Get the world and spawn a vehicles

In [2]:
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*mini*')
truck_bp = world.get_blueprint_library().filter('*firetruck*')
mini_bp = world.get_blueprint_library().filter('*cooper_s*')

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

# Get the camera and start the live stream

In [3]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)


def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))


image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image, camera_data))

# Maintain speed

In [4]:
PREFERRED_SPEED = 30
SPEED_THRESHOLD = 2


def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8  # think of it as % of "full gas"
    else:
        return 0.4

# Control the vehicle in a straight line
Muestra la imagen de la cámara con los datos de velocidad y posición del vehículo
Corrige la aceleración para mantener la velocidad deseada

In [5]:
import cv2
import math

In [6]:
org = (30, 50)
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
color = (255, 255, 255)
thickness = 1
lineType = cv2.LINE_AA

In [7]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

In [8]:
while True:
    world.tick()

    if cv2.waitKey(1) == ord('q'):
        quit = True
        break

    image = camera_data['image']
    steering_angle = 0
    v = vehicle.get_velocity()

    #La velocidad se multiplica por 3.6 para convertir de m/s a km/h (1 m/s = 3.6 km/h)(60min*60seg)
    #La velocidad es la hipotenusa de la velocidad en x y la velocidad en y
    speed = round(3.6 * math.sqrt(v.x ** 2 + v.y ** 2 + v.z ** 2), 0)

    vehicle_pos = vehicle.get_transform()

    image = cv2.putText(
        image,
        'Speed: ' + str(int(speed)) + ' kmh' +
        'Position: ' + str(vehicle_pos.location),
        org,
        font,
        fontScale,
        color,
        thickness,
        lineType
    )

    # determina la aceleración
    estimated_throttle = maintain_speed(speed)
    # aplica la aceleración y el ángulo de giro
    vehicle.apply_control(
        carla.VehicleControl(
            throttle=estimated_throttle,
            steer=steering_angle
        )
    )

    cv2.imshow('RGB Camera', image)

#clean up
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

# Draw a route

In [8]:
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [9]:

point_a = start_point.location
point_b = carla.Location(x=-640.644844, y=240.471010, z=0.600000)

grp = GlobalRoutePlanner(world.get_map(), 1)
# route = grp.trace_route(point_a, point_b)

#calcula la ruta mas larga entre todas las posibles
distance = 0
for loc in spawn_points:
    cur_route = grp.trace_route(point_a, loc.location)
    if len(cur_route)>distance:
        distance = len(cur_route)
        route = cur_route
    
#draw the route in sim window - Note it does not get into the camera of the car
for waypoint in route:
    world.debug.draw_string(
        waypoint[0].transform.location,
        '^',
        draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255),
        life_time=600.0,
        persistent_lines=True
    )

In [10]:
# draw some reference points
for reference in [25, 50, 75, 100, 125, 150, 175, 200]:
    world.debug.draw_string(
        route[reference][0].transform.location,
        str(reference),
        draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255),
        life_time=600.0,
        persistent_lines=True
    )

# calculate the angle to next waypoint

In [11]:
import math

In [12]:
# ejemplo de como calcular el angulo entre el vehiculo y el siguiente waypoint

car_x = vehicle.get_transform().location.x
car_y = vehicle.get_transform().location.y

wp_x = route[25][0].transform.location.x
wp_y = route[25][0].transform.location.y

angle = math.degrees(math.atan((wp_y - car_y) / (wp_x - car_x)))
print('angle to wp: ', angle)

angle to wp:  -39.73580371095364


In [13]:
#1. Actual direction/angle of the car
#2. Direction to next waypoint
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

In [14]:
print(
    'our angle to WP 25: ',
    angle_between((0.769, -0.639), (0.999, 0.003))
)
#examples
print(angle_between((1, 0), (0, 1)))  # between east and north
print(angle_between((1, 0), (1, 0)))  # between same direction
print(angle_between((1, 0), (-1, 0)))  # between east and west

our angle to WP 25:  -39.896921411360864
-90.0
0.0
-180.0


In [15]:
def get_angle(car, wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y

    # vector to waypoint
    x = (wp_x - car_x) / ((wp_y - car_y) ** 2 + (wp_x - car_x) ** 2) ** 0.5
    y = (wp_y - car_y) / ((wp_y - car_y) ** 2 + (wp_x - car_x) ** 2) ** 0.5

    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x, y), (car_vector.x, car_vector.y))

    return degrees

In [16]:
# little test
get_angle(vehicle, route[125][0])

-99.62705749516019

# Control the vehicle in a route with angle correction

In [18]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

PREFERRED_SPEED = 90
SPEED_THRESHOLD = 2 
MAX_STEER_DEGREES = 40
curr_wp = 5
while curr_wp < len(route) - 1:
    world.tick()

    if cv2.waitKey(1) == ord('q'):
        vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
        break

    image = camera_data['image']

    # move to next wp if we are too close
    while (curr_wp < len(route) and
           vehicle.get_transform().location.distance(route[curr_wp][0].transform.location) < 5):
        curr_wp += 1

    predicted_angle = get_angle(vehicle, route[curr_wp][0])
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x ** 2 + v.y ** 2 + v.z ** 2), 0)
    
    image = cv2.putText(
        image,
        'Steering angle: ' + str(round(predicted_angle, 3))+
        'Speed: ' + str(int(speed))+
        'Next wp: '+str(curr_wp),
        org, font, fontScale, color, thickness, lineType
    )
    
    estimated_throttle = maintain_speed(speed)
    
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle

    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
        
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75
    
    vehicle.apply_control(
        carla.VehicleControl(throttle=estimated_throttle, steer=steer_input)
    )
    cv2.imshow('RGB Camera',image)
    
#clean up
cv2.destroyAllWindows()
camera.stop()
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

KeyboardInterrupt: 

In [40]:
# utility script of destruction
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)